Experiment 6:

Training sentences: a b c a

Embedding dimension = 10 or 100

Number of words = 20

Use learnable or sinusoidal positional encodings

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as npr
import random

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from keras.optimizers import Adam
from keras_nlp.layers import PositionEmbedding

In [3]:
seed = 428

np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)

In [4]:
def bert_module(query, key, value, embed_dim, num_head, i):
    
    # Multi headed self-attention
    attention_output = layers.MultiHeadAttention(
        num_heads=num_head,
        key_dim=embed_dim // num_head,
        name="encoder_{}/multiheadattention".format(i)
    )(query, key, value, use_causal_mask=True)
    
    # Add & Normalize
    attention_output = layers.Add()([query, attention_output])  # Skip Connection
    attention_output = layers.LayerNormalization(epsilon=1e-6)(attention_output)
    
    # Feedforward network
    ff_net = keras.models.Sequential([
        layers.Dense(2 * embed_dim, activation='relu', name="encoder_{}/ffn_dense_1".format(i)),
        layers.Dense(embed_dim, name="encoder_{}/ffn_dense_2".format(i)),
    ])

    # Apply Feedforward network
    ffn_output = ff_net(attention_output)

    # Add & Normalize
    ffn_output = layers.Add()([attention_output, ffn_output])  # Skip Connection
    ffn_output = layers.LayerNormalization(epsilon=1e-6)(ffn_output)
    
    return ffn_output

In [5]:
def get_sinusoidal_embeddings(sequence_length, embedding_dim):
    position_enc = np.array([
        [pos / np.power(10000, 2. * i / embedding_dim) for i in range(embedding_dim)]
        if pos != 0 else np.zeros(embedding_dim)
        for pos in range(sequence_length)
    ])
    position_enc[1:, 0::2] = np.sin(position_enc[1:, 0::2])  # dim 2i
    position_enc[1:, 1::2] = np.cos(position_enc[1:, 1::2])  # dim 2i+1
    return tf.cast(position_enc, dtype=tf.float32)

In [6]:
N = 20 # vocab_size

vocabs = ['word_' + str(i) for i in range(N)]

vocab_map = {}
for i in range(len(vocabs)):
    vocab_map[vocabs[i]] = i
    
pairs = []

for i in vocabs:
    for j in vocabs:
        for k in vocabs:
            if i != j and i != k and j != k:
                pairs.append((i,j,k))
            
#indicator = np.random.choice([0, 1], size=len(pairs), p=[0.5, 0.5])

# pairs_train = [pairs[i] for i in range(len(indicator)) if indicator[i] == 1]
# pairs_test = [pairs[i] for i in range(len(indicator)) if indicator[i] == 0]

pairs_train = [x for x in pairs if int(x[0].split('_')[-1]) <= 9]
pairs_test = [x for x in pairs if int(x[0].split('_')[-1]) >= 10]

In [7]:
sentences_train = []
sentences_number_train = []
sentences_test = []
sentences_number_test = []

for pair in pairs_train:
    sentences_train.append([pair[0], pair[1], pair[2], pair[0]])
    sentences_number_train.append([vocab_map[pair[0]], vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    
for pair in pairs_test:
    sentences_test.append([pair[0], pair[1], pair[2], pair[0]])
    sentences_number_test.append([vocab_map[pair[0]], vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    
x_masked_train = []
y_masked_labels_train = []
x_masked_test = []
y_masked_labels_test = []

for pair in pairs_train:
    x_masked_train.append([vocab_map[pair[0]], vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    y_masked_labels_train.append([vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    
for pair in pairs_test:
    x_masked_test.append([vocab_map[pair[0]], vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    y_masked_labels_test.append([vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    
x_masked_train = np.array(x_masked_train)
y_masked_labels_train = np.array(y_masked_labels_train)
x_masked_test = np.array(x_masked_test)
y_masked_labels_test = np.array(y_masked_labels_test)

perm = np.random.permutation(len(x_masked_train))
x_masked_train = x_masked_train[perm]
y_masked_labels_train = y_masked_labels_train[perm]

In [9]:
embed_dim = 10
num_head = 2

callback = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)
inputs = layers.Input((x_masked_train.shape[1],), dtype=tf.int64)
word_embeddings = layers.Embedding(N, embed_dim, name="word_embedding")(inputs)
sinusoidal_embeddings = get_sinusoidal_embeddings(len(x_masked_train[0]), embed_dim)
encoder_output = word_embeddings + sinusoidal_embeddings

for i in range(5):
    encoder_output = bert_module(encoder_output, encoder_output, encoder_output, embed_dim, num_head, i)

encoder_output = keras.layers.Lambda(lambda x: x[:,:-1,:], name='slice')(encoder_output)
mlm_output = layers.Dense(N, name="mlm_cls", activation="softmax")(encoder_output)
mlm_model = keras.Model(inputs = inputs, outputs = mlm_output)
adam = Adam()
mlm_model.compile(loss='sparse_categorical_crossentropy', optimizer=adam)
history = mlm_model.fit(x_masked_train, y_masked_labels_train,
                        validation_split = 0.5, callbacks = [callback], 
                        epochs=2000, batch_size=5000, 
                        verbose=2)

Epoch 1/2000
1/1 - 6s - loss: 3.3491 - val_loss: 3.3085 - 6s/epoch - 6s/step
Epoch 2/2000
1/1 - 0s - loss: 3.3033 - val_loss: 3.2764 - 155ms/epoch - 155ms/step
Epoch 3/2000
1/1 - 0s - loss: 3.2703 - val_loss: 3.2499 - 181ms/epoch - 181ms/step
Epoch 4/2000
1/1 - 0s - loss: 3.2426 - val_loss: 3.2273 - 181ms/epoch - 181ms/step
Epoch 5/2000
1/1 - 0s - loss: 3.2184 - val_loss: 3.2073 - 147ms/epoch - 147ms/step
Epoch 6/2000
1/1 - 0s - loss: 3.1969 - val_loss: 3.1901 - 179ms/epoch - 179ms/step
Epoch 7/2000
1/1 - 0s - loss: 3.1780 - val_loss: 3.1755 - 168ms/epoch - 168ms/step
Epoch 8/2000
1/1 - 0s - loss: 3.1617 - val_loss: 3.1625 - 232ms/epoch - 232ms/step
Epoch 9/2000
1/1 - 0s - loss: 3.1474 - val_loss: 3.1515 - 190ms/epoch - 190ms/step
Epoch 10/2000
1/1 - 0s - loss: 3.1352 - val_loss: 3.1428 - 167ms/epoch - 167ms/step
Epoch 11/2000
1/1 - 0s - loss: 3.1255 - val_loss: 3.1360 - 173ms/epoch - 173ms/step
Epoch 12/2000
1/1 - 0s - loss: 3.1179 - val_loss: 3.1296 - 172ms/epoch - 172ms/step
Epoch 1

Epoch 99/2000
1/1 - 0s - loss: 2.4304 - val_loss: 2.4346 - 118ms/epoch - 118ms/step
Epoch 100/2000
1/1 - 0s - loss: 2.4213 - val_loss: 2.4264 - 124ms/epoch - 124ms/step
Epoch 101/2000
1/1 - 0s - loss: 2.4132 - val_loss: 2.4237 - 123ms/epoch - 123ms/step
Epoch 102/2000
1/1 - 0s - loss: 2.4101 - val_loss: 2.4248 - 110ms/epoch - 110ms/step
Epoch 103/2000
1/1 - 0s - loss: 2.4108 - val_loss: 2.4218 - 121ms/epoch - 121ms/step
Epoch 104/2000
1/1 - 0s - loss: 2.4077 - val_loss: 2.4103 - 118ms/epoch - 118ms/step
Epoch 105/2000
1/1 - 0s - loss: 2.3975 - val_loss: 2.4025 - 120ms/epoch - 120ms/step
Epoch 106/2000
1/1 - 0s - loss: 2.3898 - val_loss: 2.4043 - 121ms/epoch - 121ms/step
Epoch 107/2000
1/1 - 0s - loss: 2.3906 - val_loss: 2.4009 - 124ms/epoch - 124ms/step
Epoch 108/2000
1/1 - 0s - loss: 2.3881 - val_loss: 2.3907 - 126ms/epoch - 126ms/step
Epoch 109/2000
1/1 - 0s - loss: 2.3778 - val_loss: 2.3875 - 130ms/epoch - 130ms/step
Epoch 110/2000
1/1 - 0s - loss: 2.3746 - val_loss: 2.3870 - 132ms/

Epoch 196/2000
1/1 - 0s - loss: 2.1748 - val_loss: 2.1857 - 128ms/epoch - 128ms/step
Epoch 197/2000
1/1 - 0s - loss: 2.1733 - val_loss: 2.1844 - 130ms/epoch - 130ms/step
Epoch 198/2000
1/1 - 0s - loss: 2.1720 - val_loss: 2.1827 - 124ms/epoch - 124ms/step
Epoch 199/2000
1/1 - 0s - loss: 2.1703 - val_loss: 2.1820 - 126ms/epoch - 126ms/step
Epoch 200/2000
1/1 - 0s - loss: 2.1696 - val_loss: 2.1807 - 123ms/epoch - 123ms/step
Epoch 201/2000
1/1 - 0s - loss: 2.1683 - val_loss: 2.1785 - 125ms/epoch - 125ms/step
Epoch 202/2000
1/1 - 0s - loss: 2.1661 - val_loss: 2.1748 - 140ms/epoch - 140ms/step
Epoch 203/2000
1/1 - 0s - loss: 2.1625 - val_loss: 2.1722 - 148ms/epoch - 148ms/step
Epoch 204/2000
1/1 - 0s - loss: 2.1601 - val_loss: 2.1704 - 144ms/epoch - 144ms/step
Epoch 205/2000
1/1 - 0s - loss: 2.1583 - val_loss: 2.1688 - 120ms/epoch - 120ms/step
Epoch 206/2000
1/1 - 0s - loss: 2.1566 - val_loss: 2.1673 - 124ms/epoch - 124ms/step
Epoch 207/2000
1/1 - 0s - loss: 2.1553 - val_loss: 2.1662 - 126ms

In [10]:
acc = []
prob = []
x_test_subset = x_masked_test[np.random.choice(x_masked_test.shape[0], size=1000, replace=False)]

for sentence_number in x_test_subset:
    temp = keras.backend.function(inputs = mlm_model.layers[0].input, outputs = mlm_model.layers[-1].output) \
        (np.array(sentence_number).reshape(1,len(sentence_number)))
    temp = temp[:,-1,:]
    acc.append(1 if temp.argmax() == sentence_number[-1] else 0)
    prob.append(temp[0][sentence_number[-1]])

In [11]:
(np.mean(acc), np.mean(prob))

(0.0, 0.040708642)

In [12]:
embed_dim = 100
num_head = 2

callback = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)
inputs = layers.Input((x_masked_train.shape[1],), dtype=tf.int64)
word_embeddings = layers.Embedding(N, embed_dim, name="word_embedding")(inputs)
sinusoidal_embeddings = get_sinusoidal_embeddings(len(x_masked_train[0]), embed_dim)
encoder_output = word_embeddings + sinusoidal_embeddings

for i in range(5):
    encoder_output = bert_module(encoder_output, encoder_output, encoder_output, embed_dim, num_head, i)

encoder_output = keras.layers.Lambda(lambda x: x[:,:-1,:], name='slice')(encoder_output)
mlm_output = layers.Dense(N, name="mlm_cls", activation="softmax")(encoder_output)
mlm_model = keras.Model(inputs = inputs, outputs = mlm_output)
adam = Adam()
mlm_model.compile(loss='sparse_categorical_crossentropy', optimizer=adam)
history = mlm_model.fit(x_masked_train, y_masked_labels_train,
                        validation_split = 0.5, callbacks = [callback], 
                        epochs=2000, batch_size=5000, 
                        verbose=2)

Epoch 1/2000
1/1 - 8s - loss: 3.6666 - val_loss: 3.2988 - 8s/epoch - 8s/step
Epoch 2/2000
1/1 - 1s - loss: 3.2762 - val_loss: 3.1187 - 801ms/epoch - 801ms/step
Epoch 3/2000
1/1 - 1s - loss: 3.0928 - val_loss: 3.0142 - 768ms/epoch - 768ms/step
Epoch 4/2000
1/1 - 1s - loss: 2.9847 - val_loss: 2.9598 - 783ms/epoch - 783ms/step
Epoch 5/2000
1/1 - 1s - loss: 2.9298 - val_loss: 2.9000 - 809ms/epoch - 809ms/step
Epoch 6/2000
1/1 - 1s - loss: 2.8705 - val_loss: 2.8506 - 802ms/epoch - 802ms/step
Epoch 7/2000
1/1 - 1s - loss: 2.8221 - val_loss: 2.8194 - 688ms/epoch - 688ms/step
Epoch 8/2000
1/1 - 1s - loss: 2.7915 - val_loss: 2.7655 - 639ms/epoch - 639ms/step
Epoch 9/2000
1/1 - 1s - loss: 2.7369 - val_loss: 2.6987 - 605ms/epoch - 605ms/step
Epoch 10/2000
1/1 - 1s - loss: 2.6682 - val_loss: 2.6356 - 608ms/epoch - 608ms/step
Epoch 11/2000
1/1 - 1s - loss: 2.6027 - val_loss: 2.5779 - 542ms/epoch - 542ms/step
Epoch 12/2000
1/1 - 1s - loss: 2.5427 - val_loss: 2.5304 - 543ms/epoch - 543ms/step
Epoch 1

In [13]:
acc = []
prob = []
x_test_subset = x_masked_test[np.random.choice(x_masked_test.shape[0], size=1000, replace=False)]

for sentence_number in x_test_subset:
    temp = keras.backend.function(inputs = mlm_model.layers[0].input, outputs = mlm_model.layers[-1].output) \
        (np.array(sentence_number).reshape(1,len(sentence_number)))
    temp = temp[:,-1,:]
    acc.append(1 if temp.argmax() == sentence_number[-1] else 0)
    prob.append(temp[0][sentence_number[-1]])

In [14]:
(np.mean(acc), np.mean(prob))

(0.0, 0.0068704057)

In [15]:
x_masked_test

array([[10,  0,  1, 10],
       [10,  0,  2, 10],
       [10,  0,  3, 10],
       ...,
       [19, 18, 15, 19],
       [19, 18, 16, 19],
       [19, 18, 17, 19]])